<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Functions</a></span><ul class="toc-item"><li><span><a href="#日期类型数据处理" data-toc-modified-id="日期类型数据处理-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>日期类型数据处理</a></span></li><li><span><a href="#浮点数精度转换" data-toc-modified-id="浮点数精度转换-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>浮点数精度转换</a></span></li><li><span><a href="#regexp_extract" data-toc-modified-id="regexp_extract-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>regexp_extract</a></span></li><li><span><a href="#lit" data-toc-modified-id="lit-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>lit</a></span></li><li><span><a href="#concat" data-toc-modified-id="concat-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>concat</a></span></li><li><span><a href="#udf" data-toc-modified-id="udf-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>udf</a></span></li></ul></li><li><span><a href="#RDD" data-toc-modified-id="RDD-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>RDD</a></span><ul class="toc-item"><li><span><a href="#sample-vs-takeSample" data-toc-modified-id="sample-vs-takeSample-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>sample vs takeSample</a></span></li><li><span><a href="#sortBy-vs-sortByKey" data-toc-modified-id="sortBy-vs-sortByKey-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>sortBy vs sortByKey</a></span></li><li><span><a href="#takeOrdered" data-toc-modified-id="takeOrdered-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>takeOrdered</a></span></li><li><span><a href="#combineByKey" data-toc-modified-id="combineByKey-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>combineByKey</a></span></li></ul></li><li><span><a href="#DataFrame" data-toc-modified-id="DataFrame-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>DataFrame</a></span><ul class="toc-item"><li><span><a href="#数据读取:-read.csv" data-toc-modified-id="数据读取:-read.csv-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>数据读取: read.csv</a></span></li><li><span><a href="#数据类型转换:-cast,-astype" data-toc-modified-id="数据类型转换:-cast,-astype-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>数据类型转换: cast, astype</a></span></li><li><span><a href="#浮点数精度转换" data-toc-modified-id="浮点数精度转换-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>浮点数精度转换</a></span></li></ul></li><li><span><a href="#可视化" data-toc-modified-id="可视化-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>可视化</a></span></li></ul></div>

# Functions

## 日期类型数据处理

日期数据处理需要使用到`functions.year(), funcitons.month(), functions.day()`等相关日期处理函数

参考资料： [https://stackoverflow.com/questions/30949202/spark-dataframe-timestamptype-how-to-get-year-month-day-values-from-field](https://stackoverflow.com/questions/30949202/spark-dataframe-timestamptype-how-to-get-year-month-day-values-from-field)

In [63]:
stock_df.select(F.year('Date')).show(5)

+----------+
|year(Date)|
+----------+
|      2012|
|      2012|
|      2012|
|      2012|
|      2012|
+----------+
only showing top 5 rows



In [64]:
stock_df.select(F.dayofweek('Date')).show(5)

+---------------+
|dayofweek(Date)|
+---------------+
|              3|
|              4|
|              5|
|              6|
|              2|
+---------------+
only showing top 5 rows



## 浮点数精度转换

浮点数精度转换可以使用`format_number， round`两种方法

In [59]:
stock_df.select('Close').show(5)

+-----+
|Close|
+-----+
|60.33|
|59.71|
|59.42|
| 59.0|
|59.18|
+-----+
only showing top 5 rows



In [60]:
# format_number
stock_df.select(F.format_number('Close', 1)).show(5)

+-----------------------+
|format_number(Close, 1)|
+-----------------------+
|                   60.3|
|                   59.7|
|                   59.4|
|                   59.0|
|                   59.2|
+-----------------------+
only showing top 5 rows



可以看到`format_number`采用了四舍五入的方法

In [61]:
# round
stock_df.select(F.round('Close', 1)).show(5)

+---------------+
|round(Close, 1)|
+---------------+
|           60.3|
|           59.7|
|           59.4|
|           59.0|
|           59.2|
+---------------+
only showing top 5 rows



当传入的第二个参数大于等于0时，round函数对小数部分取精度

In [62]:
stock_df.select(F.round('Close', -1)).show(5)

+----------------+
|round(Close, -1)|
+----------------+
|            60.0|
|            60.0|
|            60.0|
|            60.0|
|            60.0|
+----------------+
only showing top 5 rows



当传入的第二个参数小于0时，round函数对整数部分四舍五入，并且保留一位小数

参考资料： [https://stackoverflow.com/questions/48832493/how-to-set-display-precision-in-pyspark-dataframe-show](https://stackoverflow.com/questions/48832493/how-to-set-display-precision-in-pyspark-dataframe-show)

## regexp_extract

`regexp_extract`用于使用给定的正则表达式，从某列字符串中提取出需要的内容并且形成DataFrame中的一列

In [69]:
df = spark.createDataFrame([('120-1456',), ('456-987', )], ['col_1'])

In [70]:
df.show()

+--------+
|   col_1|
+--------+
|120-1456|
| 456-987|
+--------+



In [77]:
extract_df = df.select(regexp_extract('col_1', r'(\d+)-(\d+)', 1).alias('first'),
                       regexp_extract('col_1', r'(\d+)-(\d+)', 2).alias('second'))

In [78]:
extract_df.show()

+-----+------+
|first|second|
+-----+------+
|  120|  1456|
|  456|   987|
+-----+------+



In [84]:
extract_df.dtypes

[('first', 'string'), ('second', 'string')]

## lit

创建一个列，这个列中每个元素都是一个传入的常量值

In [86]:
extract_df.select(F.lit('$').alias('const')).show()

+-----+
|const|
+-----+
|    $|
|    $|
+-----+



## concat

拼接两个列中的元素

In [93]:
extract_df.select(F.concat(extract_df['first'], extract_df['second']).alias('concat')).show()

+-------+
| concat|
+-------+
|1201456|
| 456987|
+-------+



## udf

执行一个用户自定义的函数

In [98]:
# 这个函数在一个字符串末尾增加下划线
def add_(s):
    return s + '_'

In [99]:
udf_add = udf(add_)

In [100]:
extract_df.select(udf_add('first').alias('upper')).show()

+-----+
|upper|
+-----+
| 120_|
| 456_|
+-----+



# RDD

## sample vs takeSample

+ `sample`是从数据中采样某个fraction的数据，参数是数据个数占比
+ `takeSample`是从数据中采样几个样本，参数是样本的个数

## sortBy vs sortByKey

+ `sortByKey`是按照字典的建值排序
+ `sortBy`可以传入自定义的比较函数，从而可以实现按照value比较大小

## takeOrdered

取出数据中的前n个元素还可以使用`takeOrdered`函数，这个函数按照给定的key获取从小到大的n个元素。如果希望按照逆序的方式排列元素，可以采用如下方法：

```python
sc.parallelize([10, 1, 2, 9, 3, 4, 5, 6, 7], 2).takeOrdered(6, key=lambda x: -x)
```

## combineByKey

`combineByKey`这个函数可以按照key值进行聚合，得到由value组成的一个list


In [41]:
x = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])

In [43]:
x.combineByKey(lambda x: [x], lambda c, v: c + [v], lambda x,y: x + y).collect()

[('b', [1]), ('a', [1, 2])]

# DataFrame

## 数据读取: read.csv

对于参数`header`，如果不设置spark默认自动加上形如`_cx`的列名。如果希望使用第一行作为表头需要设置`header`参数为True 

`spark.read.csv`函数中，可以通过给path传递一个路径的list，从而同时读取多个含有相同结构数据的文件内容到一个变量中。不需要如pandas一样，使用`pd.concat`方法合并多个DataFrame。非常的方便

## 数据类型转换: cast, astype

类型转换可以使用`cast`或者`astype`两个函数，它们是等价的。

In [51]:
stock_df.select([stock_df['Close'].astype('float')]).show(5)

+-----+
|Close|
+-----+
|60.33|
|59.71|
|59.42|
| 59.0|
|59.18|
+-----+
only showing top 5 rows



In [52]:
stock_df.select([stock_df['Close'].cast('float')]).show(5)

+-----+
|Close|
+-----+
|60.33|
|59.71|
|59.42|
| 59.0|
|59.18|
+-----+
only showing top 5 rows



## 浮点数精度转换

浮点数精度转换可以使用`format_number， round`两种方法

In [59]:
stock_df.select('Close').show(5)

+-----+
|Close|
+-----+
|60.33|
|59.71|
|59.42|
| 59.0|
|59.18|
+-----+
only showing top 5 rows



In [60]:
# format_number
stock_df.select(F.format_number('Close', 1)).show(5)

+-----------------------+
|format_number(Close, 1)|
+-----------------------+
|                   60.3|
|                   59.7|
|                   59.4|
|                   59.0|
|                   59.2|
+-----------------------+
only showing top 5 rows



可以看到`format_number`采用了四舍五入的方法

In [61]:
# round
stock_df.select(F.round('Close', 1)).show(5)

+---------------+
|round(Close, 1)|
+---------------+
|           60.3|
|           59.7|
|           59.4|
|           59.0|
|           59.2|
+---------------+
only showing top 5 rows



当传入的第二个参数大于等于0时，round函数对小数部分取精度

In [62]:
stock_df.select(F.round('Close', -1)).show(5)

+----------------+
|round(Close, -1)|
+----------------+
|            60.0|
|            60.0|
|            60.0|
|            60.0|
|            60.0|
+----------------+
only showing top 5 rows



当传入的第二个参数小于0时，round函数对整数部分四舍五入，并且保留一位小数

参考资料： [https://stackoverflow.com/questions/48832493/how-to-set-display-precision-in-pyspark-dataframe-show](https://stackoverflow.com/questions/48832493/how-to-set-display-precision-in-pyspark-dataframe-show)

# 可视化

当需要可视化时，需要将Spark的DataFrame转化为Pandas的DataFrame。将spark的DataFrame转化为pandas的DataFrame可以有两种方法：
+ `spark_df.toPandas()`
+ `pandas_df = pd.DataFrame(spark_df.rdd.map(lambda l: l.asDict()).collect())`

其中，`asDict`方法的作用是将一个`Row`转化为一个`(列名，值)`的字典。

In [35]:
start_time = time.time()
counts_pddf = pd.DataFrame(crime_type_counts.rdd.map(lambda l: l.asDict()).collect())
print(time.time() - start_time)

0.30457425117492676


In [36]:
start_time = time.time()
counts_pddf = crime_type_counts.toPandas()
print(time.time() - start_time)

0.10435199737548828


**说明** 在spark中，大量的数据分布存储在不同的节点上。当我们调用`toPandas`方法转化为panda dataFrame的时候，spark会将各个节点上的数据汇总到一起，然后再转化。这样会导致需要大量的时间和资源。因此，我们应该只是将最终需要可视化的**小**数据转化为pandas的DataFrame。并且尽可能的之前将数据缓存在内存中

[https://www.quora.com/What-is-an-efficient-way-to-convert-a-large-spark-dataframe-to-pandas](https://www.quora.com/What-is-an-efficient-way-to-convert-a-large-spark-dataframe-to-pandas)